# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# Automated Pipeline

In [4]:
# ✅ 셀 1: 라이브러리 및 설정
from datetime import datetime
import numpy as np
import pandas as pd
import joblib
import wandb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [5]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경변수 불러오기
load_dotenv()

# 환경변수에서 API 키 가져오기
WANDB_API_KEY = os.getenv("WANDB_API_KEY")

In [6]:
# ✅ 셀 1.5: wandb 로그인 (단 1회)
wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/airflow/.netrc
wandb: Currently logged in as: songhune0627 (jandar-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [21]:
# ✅ 셀 2: 설정 값 정의
HYPERPARAMS = {
    "DecisionTree": {"max_depth": 7},
    "RandomForest": {"n_estimators": 30, "max_depth": 5},
    "XGBoost": {"n_estimators": 75, "learning_rate": 0.1},
    "LightGBM": {"n_estimators": 75, "learning_rate": 0.1}
}

WANDB_PROJECT = "weather_modeling"

In [15]:
# ✅ 셀 3: 데이터 로드
df = pd.read_csv("prepared_weather_df.csv") 
X = df.drop(columns=["weather"])
y = df["weather"]

In [16]:
# ✅ 셀 3.5: 범주형 변수 인코딩 (오류 - 일시적 코드 추가!)
from sklearn.preprocessing import LabelEncoder

# 문자열(범주형) 컬럼만 추출
categorical_cols = X.select_dtypes(include=["object"]).columns

# 각 범주형 컬럼에 대해 Label Encoding
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [17]:
# ✅ LightGBM 등 오류 방지용으로 컬럼 이름 재정의
X.columns = [f"col_{i}" for i in range(X.shape[1])]

In [18]:
# ✅ 셀 4: 데이터 분할
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

In [ ]:
# ✅ 셀 5: 모델 학습 및 평가 함수 정의
def train_and_evaluate(model_name, model_class, params):
    print(f"===== {model_name} =====")

    model = model_class(**params)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=kf)
    print(f"Cross-validated RMSE: {-cv_scores.mean():.4f}")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = mean_squared_error(y_test, y_pred) ** 0.5 # Version Error - 직접 식을 입력하는 방식으로 수정
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Test RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

    # WandB 로깅
    # wandb.login(key=WANDB_API_KEY)
    run = wandb.init(project=WANDB_PROJECT, name=f"{model_name}_run", reinit=True)
    run.config.update(params)
    wandb.log({"RMSE": rmse, "MAE": mae, "R2": r2})

    model_path = f"{model_name}_model.pkl"
    joblib.dump(model, model_path)
    wandb.save(model_path)
    run.finish()

In [22]:
"""✅ 셀 5 -> K-Fold 최적값 확인 & 적용 후 학습 진행하는 방식으로 수정"""
def train_and_evaluate(model_name, model_class, params):
    print(f"===== {model_name} =====")

    # 교차 검증: RMSE (음수로 반환되므로 - 붙이기)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_rmse_scores = []
    models = []

    for i, (train_idx, val_idx) in enumerate(kf.split(X_trainval)):
        X_tr, X_val = X_trainval.iloc[train_idx], X_trainval.iloc[val_idx]
        y_tr, y_val = y_trainval.iloc[train_idx], y_trainval.iloc[val_idx]

        model = model_class(**params)
        model.fit(X_tr, y_tr)
        y_val_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_val_pred) ** 0.5
        fold_rmse_scores.append(rmse)
        models.append(model)

        print(f"Fold {i+1} RMSE: {rmse:.4f}")

    mean_rmse = np.mean(fold_rmse_scores)
    print(f"✅ Cross-validated RMSE (mean): {mean_rmse:.4f}")

    # 가장 성능 좋은 fold 모델 선택
    best_model_idx = np.argmin(fold_rmse_scores)
    best_model = models[best_model_idx]

    # 테스트셋 평가
    y_pred = best_model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"🚀 Test RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

    # WandB 로깅
    run = wandb.init(project=WANDB_PROJECT, name=f"{model_name}_run", reinit=True)
    run.config.update(params)
    wandb.log({
        "CV_RMSE": mean_rmse,
        "Test_RMSE": rmse,
        "MAE": mae,
        "R2": r2
    })

    model_path = f"{model_name}_model.pkl"
    joblib.dump(best_model, model_path)
    wandb.save(model_path)
    run.finish()

In [23]:
# ✅ 셀 6: 모델 실행
target_models = [
    ("DecisionTree", DecisionTreeRegressor, HYPERPARAMS["DecisionTree"]),
    ("RandomForest", RandomForestRegressor, HYPERPARAMS["RandomForest"]),
    ("XGBoost", XGBRegressor, HYPERPARAMS["XGBoost"]),
    ("LightGBM", LGBMRegressor, HYPERPARAMS["LightGBM"])
]

for model_name, model_class, params in target_models:
    train_and_evaluate(model_name, model_class, params)

===== DecisionTree =====
Fold 1 RMSE: 0.0000
Fold 2 RMSE: 0.0000
Fold 3 RMSE: 0.0000
Fold 4 RMSE: 0.0000
Fold 5 RMSE: 0.0000
✅ Cross-validated RMSE (mean): 0.0000
🚀 Test RMSE: 0.0000, MAE: 0.0000, R2: 1.0000


CV_RMSE,▁
MAE,▁
R2,▁
Test_RMSE,▁
CV_RMSE,0
MAE,0
R2,1
Test_RMSE,0


===== RandomForest =====
Fold 1 RMSE: 0.0000
Fold 2 RMSE: 0.0000
Fold 3 RMSE: 0.0000
Fold 4 RMSE: 0.0000
Fold 5 RMSE: 0.0000
✅ Cross-validated RMSE (mean): 0.0000
🚀 Test RMSE: 0.0000, MAE: 0.0000, R2: 1.0000


CV_RMSE,▁
MAE,▁
R2,▁
Test_RMSE,▁
CV_RMSE,0
MAE,0
R2,1
Test_RMSE,0


===== XGBoost =====
Fold 1 RMSE: 0.0000
Fold 2 RMSE: 0.0000
Fold 3 RMSE: 0.0000
Fold 4 RMSE: 0.0000
Fold 5 RMSE: 0.0000
✅ Cross-validated RMSE (mean): 0.0000
🚀 Test RMSE: 0.0000, MAE: 0.0000, R2: 1.0000


CV_RMSE,▁
MAE,▁
R2,▁
Test_RMSE,▁
CV_RMSE,0
MAE,0
R2,1
Test_RMSE,0


===== LightGBM =====
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11223
[LightGBM] [Info] Number of data points in the train set: 63732, number of used features: 61
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

CV_RMSE,▁
MAE,▁
R2,▁
Test_RMSE,▁
CV_RMSE,0
MAE,0
R2,1
Test_RMSE,0
